In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#Loading the dataset
df = pd.read_csv('mlc.csv')

In [51]:
print("Top 5 batsmen by total runs:")
batsmen_runs = df.groupby('striker')['runs_off_bat'].sum().sort_values(ascending=False).head(5)
print(batsmen_runs)
print("\n")

Top 5 batsmen by total runs:
striker
F du Plessis    420
SPD Smith       336
TM Head         336
FH Allen        306
DP Conway       293
Name: runs_off_bat, dtype: int64




In [53]:
print("Batsmen with most centuries:")

# Group by match_id and striker, sum the runs scored by each batsman in each match
batsman_match_runs = df.groupby(['match_id', 'striker'])['runs_off_bat'].sum()

# Count matches where a batsman scored 100 or more runs
centuries = batsman_match_runs[batsman_match_runs >= 100].reset_index()
centuries_count = centuries.groupby('striker').size().sort_values(ascending=False)

# Batsmen with most centuries
print(centuries_count.head(3))
print("\n")

Batsmen with most centuries:
striker
F du Plessis    1
FH Allen        1
RD Rickelton    1
dtype: int64




In [54]:
# Find batsmen with most ducks (scoring zero in an innings)
print("Batsmen with most ducks:")

# Group by match_id and striker to get runs per innings
batsman_match_runs = df.groupby(['match_id', 'striker'])['runs_off_bat'].sum().reset_index()

# Filter for innings where batsman scored zero runs
ducks = batsman_match_runs[batsman_match_runs['runs_off_bat'] == 0]

# Count ducks for each batsman
duck_counts = ducks.groupby('striker').size().sort_values(ascending=False)

# Display batsmen with most ducks
print(duck_counts.head(5))
print("\n")

Batsmen with most ducks:
striker
RD Rickelton       2
Aaron Jones        1
C le Roux          1
TM Head            1
Shakib Al Hasan    1
dtype: int64




In [55]:
print("Number of wickets taken by each bowler:")
# Excluding Run Outs
bowler_wicket_types = ['bowled', 'caught', 'lbw', 'stumped', 'caught and bowled', 'hit wicket']
bowler_credited_wickets = df[
    (df['wicket_type'].notna()) & 
    (~df['wicket_type'].isin(['run out']))
]
# Counting wickets by bowler
bowler_wickets = bowler_credited_wickets.groupby('bowler').size().sort_values(ascending=False)
print(bowler_wickets.head(10))
print("\n")

Number of wickets taken by each bowler:
bowler
SN Netravalkar    15
R Ravindra        12
M Jansen          11
CJ Gannon         11
GJ Maxwell        10
LH Ferguson       10
Hassan Khan       10
Rashid Khan       10
SH Johnson         9
Noor Ahmad         9
dtype: int64




In [56]:
print("Count of different types of dismissals:")
dismissal_counts = df['wicket_type'].value_counts()
print(dismissal_counts)
print("\n")

Count of different types of dismissals:
caught               179
bowled                53
lbw                   24
run out               10
caught and bowled      9
stumped                4
hit wicket             1
retired hurt           1
Name: wicket_type, dtype: int64




In [57]:
#Average runs scored in each innings
print("Average runs scored in each innings:")
innings_runs = df.groupby(['match_id', 'innings'])[['runs_off_bat', 'extras']].sum()
innings_runs['total_runs'] = innings_runs['runs_off_bat'] + innings_runs['extras']
average_by_innings = innings_runs.groupby('innings')['total_runs'].mean()
print(average_by_innings)
print("\n")

Average runs scored in each innings:
innings
1    161.173913
2    137.608696
Name: total_runs, dtype: float64




In [58]:
print("Number of dot balls by each bowler:")
dot_balls = df[(df['runs_off_bat'] == 0) & (df['extras'] == 0)].groupby('bowler').size().sort_values(ascending=False)
print(dot_balls.head(10))
print("\n")

Number of dot balls by each bowler:
bowler
M Jansen          91
LH Ferguson       79
SN Netravalkar    78
Rashid Khan       70
GJ Maxwell        65
TA Boult          65
SP Narine         61
SH Johnson        58
Zia-ul-Haq        57
Haris Rauf        55
dtype: int64




In [59]:
print("Count of boundaries by each batsman:")
fours = df[df['runs_off_bat'] == 4].groupby('striker').size()
sixes = df[df['runs_off_bat'] == 6].groupby('striker').size()
boundaries = pd.DataFrame({'Fours': fours, 'Sixes': sixes}).fillna(0).astype(int)
boundaries['Total_Boundaries'] = boundaries['Fours'] + boundaries['Sixes']
print(boundaries.sort_values('Total_Boundaries', ascending=False).head(10))
print("\n")

Count of boundaries by each batsman:
              Fours  Sixes  Total_Boundaries
striker                                     
F du Plessis     43     23                66
TM Head          40     18                58
FH Allen         27     20                47
SPD Smith        23     21                44
DP Conway        25      7                32
RD Rickelton     19     12                31
Hassan Khan      13     13                26
Rashid Khan      12     13                25
MW Short          9     14                23
N Pooran         18      5                23




In [60]:
print("Total runs scored in each over:")
# Creating Over Column
df['over'] = np.ceil(df['ball']).astype(int)
over_runs = df.groupby('over')[['runs_off_bat', 'extras']].sum()
over_runs['total_runs'] = over_runs['runs_off_bat'] + over_runs['extras']
print(over_runs[['total_runs']])
print("\n")

Total runs scored in each over:
      total_runs
over            
1            289
2            282
3            425
4            444
5            414
6            457
7            327
8            314
9            335
10           314
11           403
12           324
13           373
14           327
15           371
16           342
17           280
18           321
19           283
20           247




In [61]:
print("Powerplay vs non-powerplay scoring rates:")
df['is_powerplay'] = df['over'] <= 6
powerplay_runs = df[df['is_powerplay']].groupby(['match_id', 'innings'])[['runs_off_bat', 'extras']].sum()
powerplay_runs['total_runs'] = powerplay_runs['runs_off_bat'] + powerplay_runs['extras']
powerplay_balls = df[df['is_powerplay']].groupby(['match_id', 'innings']).size()
powerplay_rate = (powerplay_runs['total_runs'] / powerplay_balls * 6).mean()  

non_powerplay_runs = df[~df['is_powerplay']].groupby(['match_id', 'innings'])[['runs_off_bat', 'extras']].sum()
non_powerplay_runs['total_runs'] = non_powerplay_runs['runs_off_bat'] + non_powerplay_runs['extras']
non_powerplay_balls = df[~df['is_powerplay']].groupby(['match_id', 'innings']).size()
non_powerplay_rate = (non_powerplay_runs['total_runs'] / non_powerplay_balls * 6).mean()  

print(f"Powerplay scoring rate: {powerplay_rate:.2f} runs per over")
print(f"Non-powerplay scoring rate: {non_powerplay_rate:.2f} runs per over")

Powerplay vs non-powerplay scoring rates:
Powerplay scoring rate: 8.13 runs per over
Non-powerplay scoring rate: 7.96 runs per over


In [62]:
print("Top batting partnerships (by runs):")
df['partnership'] = df['striker'] + " & " + df['non_striker']
partnership_runs = df.groupby('partnership')['runs_off_bat'].sum().sort_values(ascending=False)
print(partnership_runs.head(10))
print("\n")

Top batting partnerships (by runs):
partnership
F du Plessis & DP Conway      305
TM Head & SPD Smith           208
DP Conway & F du Plessis      151
FH Allen & MW Short           147
MW Short & FH Allen           124
SPD Smith & TM Head           119
AM Hardie & DP Conway         108
FH Allen & J Fraser-McGurk     99
RD Rickelton & Q de Kock       99
TM Head & AGS Gous             93
Name: runs_off_bat, dtype: int64




In [63]:
print("Aggregate by venue:")
venue_runs = df.groupby(['match_id', 'venue'])[['runs_off_bat', 'extras']].sum()
venue_runs['total_runs'] = venue_runs['runs_off_bat'] + venue_runs['extras']
average_by_venue = venue_runs.groupby('venue')['total_runs'].mean().sort_values(ascending=False)
print(average_by_venue)
print("\n")

Aggregate by venue:
venue
Grand Prairie Stadium, Dallas      317.0625
Church Street Park, Morrisville    257.0000
Name: total_runs, dtype: float64




In [64]:
print("Top batsman-bowler matchups (min 5 balls faced):")
# Group by batsman-bowler combinations
matchups = df.groupby(['striker', 'bowler']).agg(
    balls=pd.NamedAgg(column='ball', aggfunc='count'),
    runs=pd.NamedAgg(column='runs_off_bat', aggfunc='sum'),
    dismissals=pd.NamedAgg(column='wicket_type', aggfunc=lambda x: x.notna().sum())
)

#Calculate strike rate
matchups['strike_rate'] = (matchups['runs'] / matchups['balls']) * 100

# Filter for minimum 10 balls faced
qualified_matchups = matchups[matchups['balls'] >= 5].sort_values('runs', ascending=False)
print(qualified_matchups.head(15))
print("\n")

Top batsman-bowler matchups (min 5 balls faced):
                                 balls  runs  dismissals  strike_rate
striker          bowler                                              
GJ Maxwell       Hassan Khan        19    47           0   247.368421
RD Rickelton     SH Johnson         26    46           1   176.923077
SPD Smith        J Drysdale         25    44           0   176.000000
F du Plessis     TA Boult           19    43           0   226.315789
FH Allen         Zia-ul-Haq         21    43           1   204.761905
F du Plessis     M Jansen           17    41           0   241.176471
TM Head          C le Roux          20    40           0   200.000000
SPD Smith        Haris Rauf         19    36           0   189.473684
DP Conway        Haris Rauf         20    34           1   170.000000
SP Krishnamurthi SN Netravalkar     19    32           1   168.421053
RD Rickelton     CA Dry             18    32           0   177.777778
N Pooran         CJ Gannon          14   

In [75]:
print("Golden Arm' Bowlers - Partnership BreakerS")

if 'over' not in df.columns:
    df['over'] = np.ceil(df['ball']).astype(int)

wicket_count = df['wicket_type'].notna().sum()
print(f"Total wickets in dataset: {wicket_count}")

# A simpler approach: identify all wickets and the runs scored since last wicket
df = df.sort_values(['match_id', 'innings', 'ball'])

# Group by match and innings
def find_partnership_breakers(group):
    group['last_wicket_ball'] = float('nan')
    group['partnership_runs'] = 0
    
    last_wicket_ball = 0
    current_runs = 0
    
    for idx, row in group.iterrows():
        # Add current ball's runs to partnership
        current_runs += row['runs_off_bat'] + row['extras']
        
        # If this is a wicket ball
        if pd.notna(row['wicket_type']):
            # Mark the partnership runs for this wicket
            group.at[idx, 'partnership_runs'] = current_runs
            current_runs = 0
    
    return group

# Apply function to each match-innings group
result = df.groupby(['match_id', 'innings']).apply(find_partnership_breakers)

# Analyze partnership breakers - lower threshold to 15 runs
min_partnership = 30  # Lower threshold to ensure we get results
partnership_breakers = result[pd.notna(result['wicket_type']) & (result['partnership_runs'] >= min_partnership)]

# Count how many significant partnerships each bowler broke
if len(partnership_breakers) > 0:
    golden_arm = partnership_breakers.groupby('bowler').size().sort_values(ascending=False)
    golden_arm_df = pd.DataFrame({'partnership_breaks': golden_arm})

    # Add average partnership runs when breaking
    partnership_sizes = partnership_breakers.groupby('bowler')['partnership_runs'].mean()
    golden_arm_df['avg_partnership_size'] = partnership_sizes
    
    # Only include bowlers who broke at least 3 partnerships
    golden_arm_df = golden_arm_df[golden_arm_df['partnership_breaks'] >= 3]
    
    print("\nTop 'Golden Arm' Bowlers:")
    print(golden_arm_df.head(10))
else:
    print("No significant partnerships were broken based on current criteria.")

Golden Arm' Bowlers - Partnership BreakerS
Total wickets in dataset: 281

Top 'Golden Arm' Bowlers:
             partnership_breaks  avg_partnership_size
bowler                                               
MP Stoinis                    5             52.400000
Noor Ahmad                    5             54.000000
Haris Rauf                    4             34.750000
LE Plunkett                   3             55.666667


C:\Users\aadit\AppData\Local\Temp\ipykernel_41072\4261915946.py:33: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  result = df.groupby(['match_id', 'innings']).apply(find_partnership_breakers)


In [66]:
# Find highest individual score in an innings
print("Highest individual scores in an innings:")

# Group by match_id and striker, sum the runs scored by each batsman in each match
batsman_match_runs = df.groupby(['match_id', 'striker'])['runs_off_bat'].sum().reset_index()

# Sort by runs in descending order
highest_scores = batsman_match_runs.sort_values(by='runs_off_bat', ascending=False)

# Display top 10 highest individual scores
print(highest_scores.head(10))
print("\n")

Highest individual scores in an innings:
     match_id           striker  runs_off_bat
78    1432731      RD Rickelton           103
331   1432749          FH Allen           101
68    1432730      F du Plessis           100
195   1432740      RD Rickelton            89
357   1432750         SPD Smith            88
275   1432745  SP Krishnamurthi            79
236   1432743          FH Allen            77
326   1432748           TM Head            77
302   1432747      F du Plessis            72
75    1432731            JJ Roy            69




In [76]:

# Basic venue comparison analysis
def analyze_venues(df):
    # 1. Run-scoring comparison between venues
    venue_runs = df.groupby('venue').agg({
        'runs_off_bat': 'sum',
        'ball': 'count',
        'match_id': 'nunique'
    }).reset_index()
    
    # Calculate runs per ball and runs per match
    venue_runs['runs_per_ball'] = venue_runs['runs_off_bat'] / venue_runs['ball']
    venue_runs['runs_per_match'] = venue_runs['runs_off_bat'] / venue_runs['match_id']
    
    # 2. Team performance at each venue
    team_venue_performance = df.groupby(['venue', 'batting_team']).agg({
        'runs_off_bat': 'sum',
        'ball': 'count',
        'match_id': 'nunique'
    }).reset_index()
    
    team_venue_performance['runs_per_ball'] = team_venue_performance['runs_off_bat'] / team_venue_performance['ball']
    
    # 3. Wicket types comparison between venues
    wicket_types = df[df['wicket_type'].notna() & (df['wicket_type'] != '')].groupby(['venue', 'wicket_type']).size().reset_index(name='count')
    
    # 4. Extras comparison between venues
    extras_by_venue = df.groupby('venue').agg({
        'extras': 'sum',
        'wides': 'sum',
        'noballs': 'sum',
        'byes': 'sum',
        'legbyes': 'sum'
    }).reset_index()
    
    return venue_runs, team_venue_performance, wicket_types, extras_by_venue

# Main function to run the analysis
def main():
    venue_runs, team_venue_performance, wicket_types, extras_by_venue = analyze_venues(df)

    print("VENUE COMPARISON ANALYSIS")
    print("-" * 50)
    
    print("\n1. OVERALL SCORING RATES:")
    print(venue_runs[['venue', 'runs_per_ball', 'runs_per_match']])
    
    print("\n2. TEAM PERFORMANCE BY VENUE:")
    for venue in df['venue'].unique():
        print(f"\nAt {venue}:")
        venue_data = team_venue_performance[team_venue_performance['venue'] == venue]
        print(venue_data[['batting_team', 'runs_per_ball']].sort_values('runs_per_ball', ascending=False))
    
    print("\n3. DISMISSAL PATTERNS BY VENUE:")
    wicket_summary = wicket_types.pivot_table(index='venue', columns='wicket_type', values='count', fill_value=0)
    print(wicket_summary)
    
    print("\n4. EXTRAS COMPARISON:")
    print(extras_by_venue)

if __name__ == "__main__":
    main()

VENUE COMPARISON ANALYSIS
--------------------------------------------------

1. OVERALL SCORING RATES:
                             venue  runs_per_ball  runs_per_match
0  Church Street Park, Morrisville       1.178918      242.857143
1    Grand Prairie Stadium, Dallas       1.287891      301.125000

2. TEAM PERFORMANCE BY VENUE:

At Church Street Park, Morrisville:
                batting_team  runs_per_ball
4          Texas Super Kings       1.471429
5         Washington Freedom       1.310734
2     San Francisco Unicorns       1.163180
1                MI New York       1.141553
0  Los Angeles Knight Riders       1.112069
3              Seattle Orcas       1.021918

At Grand Prairie Stadium, Dallas:
                 batting_team  runs_per_ball
11         Washington Freedom       1.444284
8      San Francisco Unicorns       1.418011
10          Texas Super Kings       1.331529
6   Los Angeles Knight Riders       1.194726
9               Seattle Orcas       1.125506
7                